In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
from sklearn.ensemble import RandomForestClassifier

class MultiNomialNB:
    def __init__(self):
        self.thetak = defaultdict(float)
        # self.thetajk = defaultdict(partial(np.ndarray, 0))
        self.thetajk= []
        self.classes = []
        
    def fit(self, X, y):
        self.classes = y.unique()
        self.thetajk = np.arange(len(self.classes)*X.shape[1], dtype=float).reshape(len(self.classes),X.shape[1])

        for i,k in enumerate(self.classes):
            self.thetak[k] = (y==k).sum()/float(y.shape[0])
            indices = np.where(y==k)[0]
            filteredX = X[indices]
            self.thetajk[i] = (filteredX.sum(axis=0)+0.01)/float(filteredX.shape[0]+0.02)
            self.thetajk[i] = self.thetajk[i]
        print(self.thetak, self.thetajk)
    
    def predict(self, X):
        left = X.dot(np.log(self.thetajk).T)
        right = (1-X.toarray()).dot(np.log(1-self.thetajk).T)
        temp_sum = np.add(left, right)
        max_num = np.argmax(temp_sum,axis=1)
        return self.classes[max_num]

    def score(self, X, y):
        y_pred = self.predict(X)
        
        score = (y_pred==y).sum()/float(y.shape[0])
        return score

if __name__ == "__main__":
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
    from sklearn import preprocessing
    from sklearn.model_selection import GridSearchCV
    
    redditData = pd.read_csv("../data/reddit_train.csv") #, sep="\n", header=None) 
    redditDataTest = pd.read_csv("../data/reddit_test.csv") # sep="\n", header=None)

    comments = redditData.iloc[:,1]
    subreddits = redditData.iloc[:,-1]
    comments = [str (item) for item in comments]
    
    tf = TfidfVectorizer(max_features=1000, stop_words='english')
    comments = tf.fit_transform(comments).toarray()
    le = preprocessing.LabelEncoder()
    subreddits = le.fit_transform(subreddits)
    print(comments.shape)
    print(subreddits)
#     x_train, x_test, y_train, y_test = train_test_split(comments, subreddits, test_size=0.2, random_state=4)
    
    print("ready")
    
    estimator = RandomForestClassifier()
    param_grid = { 
            "n_estimators": [10,20,30],
            "max_features": ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            "max_depth":[2,4,6,8,9],
            }

    grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

    grid.fit(comments, subreddits)

    print(grid.best_score_)
    print(grid.best_params_)
#     classifier = RandomForestClassifier(n_estimators=10, random_state=0)
#     classifier.fit(x_train, y_train)
#     classifier.score(x_test, y_test)
    
#     from keras.models import Sequential
    
#     model = Sequential()
#     model.add(Embedding(1000, 100, input_length=x_train.shape[1]))
#     model.add(SpatialDropout1D(0.2))
#     model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#     model.add(Dense(13, activation='softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#     epochs = 5
#     batch_size = 64

#     history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
    
#     accr = model.evaluate(X_test,Y_test)
#     print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
#     cmultiNB = MultiNomialNB()
#     cmultiNB.fit(x_train_v, y_train)

#     print(cmultiNB.score(x_test_v, y_test))

(70000, 1000)
[11 14 12 ...  6  4  1]
ready


In [4]:
print(classifier.score(x_test, y_test))

0.2681428571428571


In [4]:
!pip uninstall keras y

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
^C
Operation cancelled by user


In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import numbers
import decimal
import scipy.stats as ss
import matplotlib.pyplot as plt
import time
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ethanitovitch/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ethanitovitch/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
import string

redditDataTrain = pd.read_csv("../data/reddit_train.csv") #, sep="\n", header=None) 
commentsTrain = redditDataTrain.iloc[:,1]
subredditsTrain = redditDataTrain.iloc[:,-1]

def remove(text):
  return "".join([c for c in text if c not in string.punctuation])

commentsTrain = commentsTrain.apply(lambda x: remove(x))

tokenizer = RegexpTokenizer(r'\w+')
commentsTrain = commentsTrain.apply(lambda x: tokenizer.tokenize(x.lower()))

def stop(text):
  return [w for w in text if w not in stopwords.words('english')]

commentsTrain = commentsTrain.apply(lambda x: stop(x))

lemmatizer = WordNetLemmatizer()

def lem(text):
  return [lemmatizer.lemmatize(i) for i in text]

commentsTrain = commentsTrain.apply(lambda x: lem(x))

stem = PorterStemmer()
def stemmer(text):
  return " ".join([stem.stem(i) for i in text])

commentsTrain = commentsTrain.apply(lambda x: stemmer(x))

/Users/ethanitovitch/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:16: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 6: ordinal not in range(128)

In [ ]:
GX_train = commentsTrain[commentsTrain.index < np.percentile(commentsTrain.index, 80)].sort_index()
Gy_train = subredditsTrain[subredditsTrain.index < np.percentile(subredditsTrain.index, 80)].sort_index().values
GX_test = commentsTrain[commentsTrain.index > np.percentile(commentsTrain.index, 80)].sort_index()
Gy_test = subredditsTrain[subredditsTrain.index > np.percentile(subredditsTrain.index, 80)].sort_index().values

tfidf = TfidfVectorizer(stop_words='english')
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10000)
GX_test_idf = pca.fit(GX_test_idf.toarray())

clf = MultinomialNB(alpha=0.31005)
estimator.fit(GX_train_idf,Gy_train)
estimator.score(GX_test_idf, Gy_test)